In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, col, udf, StringType

In [2]:
spark = SparkSession.builder.appName("CleaningApp").getOrCreate()

24/06/08 14:37:46 WARN Utils: Your hostname, boui resolves to a loopback address: 127.0.1.1; using 192.168.0.6 instead (on interface wlp1s0)
24/06/08 14:37:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/06/08 14:37:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
# raw data 품목에 따라서 file_path설정
rawdata_file_path_bunjang = "hdfs://localhost:9000/user/test/iphone14_bunjang.csv"
rawdata_file_path_joongnaCafe = "hdfs://localhost:9000/user/test/iphone14_joongnaCafe.csv"
rawdata_file_path_joongnaSite = "hdfs://localhost:9000/user/test/iphone14_joongnaSite.csv"
rawdata_file_path_daangn = "hdfs://localhost:9000/user/test/iphone_daangn.csv"

df_bunjang = spark.read.csv(rawdata_file_path_bunjang, header = True, multiLine=True, quote='"', escape='"')
df_joongnaCafe = spark.read.csv(rawdata_file_path_joongnaCafe, header = True, multiLine=True, quote='"', escape='"')
df_joongnaSite = spark.read.csv(rawdata_file_path_joongnaSite, header = True, multiLine=True, quote='"', escape='"')
df_daangn = spark.read.csv(rawdata_file_path_daangn , header = True, multiLine=True, quote='"', escape='"')

#bunjang만 condition column이 추가로 있어서 삭제
df_bunjang = df_bunjang.drop(df_bunjang.condition)

#3사 rawdata 통합
df = df_bunjang.union(df_joongnaCafe).union(df_joongnaSite).union(df_daangn)

In [15]:
# 몇 년전, 몇 달전, 몇 주전 3가지 경우 몇 일 거래인지 정확한 판독불가
def filtering_date(upload_date):
    if upload_date is None:
        return None
    
    if '00' in upload_date:
        # 2023년 거래인 것은 확실, 월 일 부정확한 데이터, 0023년은 2023년 1년치 계산시에만 사용
        if '00.00' in upload_date:
            return '0023.01.01'
        # 년 월은 분명하지만 일은 부정확한 데이터 분류, 102X년 0X월 01일은 202X년 0X월 데이터 분석시에만 사용
        return upload_date.replace('00','01').replace('20','10')
    return upload_date
        
filtering_date_udf = udf(filtering_date, StringType())

In [16]:
df = df.withColumn('upload_date', filtering_date_udf(df.upload_date))

In [17]:
print(df.show())
print(df.count())

Traceback (most recent call last):
  File "/home/boui/Desktop/github/PAUG/venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/home/boui/Desktop/github/PAUG/venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/home/boui/Desktop/github/PAUG/venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/home/boui/Desktop/github/PAUG/venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


+---------------------------------+----------------------------------+-------+-----------+-----------------------------+------+--------------------+--------------------+
|                            title|                           context|  price|upload_date|                     location|status|              imgUrl|                 url|
+---------------------------------+----------------------------------+-------+-----------+-----------------------------+------+--------------------+--------------------+
|중고폰매입 아이폰매입 아이폰15...|      👉 아이폰 최고가 매입 👈
...| 555000| 0023.01.01|    인천광역시 부평구 부평1동|     0|https://media.bun...|https://m.bunjang...|
|아이폰매입 아이폰11프로 아이폰...|  안녕하세요!
번개장터 7년차 중...| 970000| 1024.05.01|    인천광역시 부평구 부평1동|     0|https://media.bun...|https://m.bunjang...|
|      아이폰 14 pro 프로 투명 ...|    새상품입니다
1) 4개에 일괄 ...|   4000| 2024.06.05|                            -|     0|https://media.bun...|https://m.bunjang...|
|        케이스티파이 아이폰14프로| 왼쪽은 팔렸고 오른쪽 고양이만 ...|  50000| 2024.06.05| 

In [18]:
df = df.withColumn('price', df.price.cast("int")) \
    .withColumn('status', df.status.cast('int')) \
    .withColumn('upload_date', to_date(df.upload_date, 'yyyy.MM.dd'))

In [19]:
print(df.show())
print(df.count())

Traceback (most recent call last):
  File "/home/boui/Desktop/github/PAUG/venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/home/boui/Desktop/github/PAUG/venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/home/boui/Desktop/github/PAUG/venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/home/boui/Desktop/github/PAUG/venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


+---------------------------------+----------------------------------+-------+-----------+-----------------------------+------+--------------------+--------------------+
|                            title|                           context|  price|upload_date|                     location|status|              imgUrl|                 url|
+---------------------------------+----------------------------------+-------+-----------+-----------------------------+------+--------------------+--------------------+
|중고폰매입 아이폰매입 아이폰15...|      👉 아이폰 최고가 매입 👈
...| 555000| 0023-01-01|    인천광역시 부평구 부평1동|     0|https://media.bun...|https://m.bunjang...|
|아이폰매입 아이폰11프로 아이폰...|  안녕하세요!
번개장터 7년차 중...| 970000| 1024-05-01|    인천광역시 부평구 부평1동|     0|https://media.bun...|https://m.bunjang...|
|      아이폰 14 pro 프로 투명 ...|    새상품입니다
1) 4개에 일괄 ...|   4000| 2024-06-05|                            -|     0|https://media.bun...|https://m.bunjang...|
|        케이스티파이 아이폰14프로| 왼쪽은 팔렸고 오른쪽 고양이만 ...|  50000| 2024-06-05| 

In [20]:
df.printSchema()

root
 |-- title: string (nullable = true)
 |-- context: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- upload_date: date (nullable = true)
 |-- location: string (nullable = true)
 |-- status: integer (nullable = true)
 |-- imgUrl: string (nullable = true)
 |-- url: string (nullable = true)



## 이상치 제거

In [13]:
# # case 판매글 제거하기 위해 price값에서 이상치 제거
# quantiles = df.approxQuantile("price", [0.25, 0.75], 0.05)
# Q1 = quantiles[0]
# Q3 = quantiles[1]
# IQR = Q3 - Q1

# # IQR에 따라 이상치 제거
# lower_bound = Q1 - 1.5 * IQR
# upper_bound = Q3 + 1.5 * IQR

# case게시글이 많아서 case 판매 가격이 이상치로 분류가 안됨 가격 수동설정 필요
df = df.filter((col("price") >= 200000) & (col("price") <= 3000000))

In [21]:
# 중복 게시글 제거
df = df.filter(col("upload_date") != "None")
df = df.filter(~(col('context').rlike('업체|매입|최저가|최고가')))
df = df.dropDuplicates(["title", "context"])
print(df.count())

0


In [15]:
df = df.toPandas()
df.to_csv("/home/boui/Desktop/usedGoods_data/rawdata/iphone14_filtering.csv", index = False, header = True, encoding = 'utf-8-sig')